# Download SQuAD Dataset and preprocess
- Download Train + eval
- tokenize data and write to separate files (context, question, answers)

## Download

In [39]:
import importlib
import preprocess
importlib.reload(preprocess)
from preprocess import download_squad_dataset, process_split, write_to_files

train, eval = download_squad_dataset()

In [40]:
print(len(train["data"]))
print(len(eval["data"]))

442
48


## Tokenization

- tokenization function (Stanford coreNLP tokenizer python only alternative)
- mapping function: (context, context_tokens) -> dictionary mapping char indices to tokens: <br>
example ("this is a test", [this, is, a, test]) ---> 0,1,2,3 -> ("this",0), 5,6 -> ("is",1), ... etc. 

In [ ]:
import stanza

nlp = stanza.Pipeline(lang = "en", processors="tokenize", tokenize_pretokenized = False)
eval_dataset = process_split(eval, nlp)
e_context_tokens, e_question_tokens, e_answer_tokens, e_span_tokens = write_to_files(eval_dataset, "eval")

In [ ]:
#train_dataset = process_split(train)
e_context_tokens, e_question_tokens, e_answer_tokens, e_span_tokens = write_to_files(eval_dataset, "eval")
#t_context_tokens, t_question_tokens, t_answer_tokens, t_span_tokens = write_to_files(train_dataset, "train")

## Map tokens to embedding indices

- load GloVe embeddings
- map vocabulary to embedding indices


In [ ]:
import numpy as np
import os 

glove_path = os.path.abspath(os.path.dirname(os.getcwd())) + "/glove_embeddings/glove.840B.300d.txt"
print(glove_path) 
assert os.path.exists(glove_path), ("glove embeddings file missing!")
embedding_index = {}
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        vals = line.split(' ')
        word = vals[0]
        coefs = np.asarray(vals[1:], dtype='float32')
        embedding_index[word] = coefs

print("Done! ", len(embedding_index),"words loaded")

/home/luca/workspace/dnlp2025/glove_embeddings/glove.840B.300d.txt


In [ ]:
word2idx = {}
idx2word = []
embedding_dim = 300
embedding_matrix = []

word2idx["[PAD]"] = 0
word2idx["[UNK]"] = 1
idx2word.append("[PAD]")
idx2word.append("[UNK]")
embedding_matrix.append(np.zeros(embedding_dim, dtype='float32'))
embedding_matrix.append(np.zeros(embedding_dim, dtype='float32'))

def get_or_create_index(token):
    token_lower = token.lower()
    if(token_lower) in word2idx:
        return word2idx[token_lower]
    else:
        idx = len(word2idx)
        word2idx[token_lower] = idx
        idx2word.append(token_lower)
        if token_lower in embedding_index:
            embedding_matrix.append(embedding_index[token_lower])
        else:
            embedding_matrix.append(np.random.normal(scale=0.01, size=embedding_dim))
        return idx

NameError: name 'np' is not defined

: 

In [146]:
sen_idxs = []
#do this for every token in contexts,question and answers
all_tokens = []
all_tokens.extend(context_tokens)
all_tokens.extend(question_tokens)
all_tokens.extend(answer_tokens)
print(len(all_tokens))
i = 0
for tokens in all_tokens:
    if tokens is None:
        continue
    idx = [get_or_create_index(t) for t in tokens.split()]
    sen_idxs.append(idx)
print(len(sen_idxs))
print(len(word2idx))
print(len(idx2word))
print(sen_idxs[0])

31338
31338
25764
25764
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 12, 15, 8, 16, 17, 18, 19, 20, 12, 21, 22, 23, 12, 7, 8, 24, 17, 25, 19, 13, 26, 27, 28, 12, 15, 8, 24, 17, 29, 19, 13, 30, 31, 32, 33, 34, 10, 35, 36, 37, 2, 3, 38, 23, 12, 9, 5, 39, 40, 41, 42, 43, 44, 43, 45, 46, 47, 48, 12, 49, 50, 51, 52, 45, 53, 54, 43, 55, 23, 56, 57, 5, 12, 58, 2, 3, 43, 12, 16, 59, 12, 60, 61, 62, 60, 63, 64, 65, 66, 67, 68, 43, 56, 69, 56, 70, 71, 12, 72, 14, 73, 74, 2, 3, 9, 63, 75, 76, 17, 77, 78, 12, 9, 79, 80, 81, 82, 56, 60, 2, 3, 83, 60, 19, 43, 84, 85, 12, 86, 87, 88, 89, 12, 90, 76, 4, 23]


In [138]:
embedding_matrix = np.array(embedding_matrix, dtype='float32')
embedding_matrix.shape

(25764, 300)

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, hidden_dim, embedding_matrix, dropout_ratio):
        super().__init__()
        self.hidden_dim = hidden_dim
        vocab_size, embedding_dim = embedding_matrix.shape
        embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float)
        
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 1, batch_first=True, dropout=dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

        self.w = nn.Linear(hidden_dim, hidden_dim)
        self.b = nn.Parameter(torch.zeros(hidden_dim))

        self.sentinel = nn.Parameter(torch.randn(1,hidden_dim))

    def encode_sequence(self, idxs, mask):
        lengths = mask.sum(dim=1)  # [batch]
        sorted_lens, sorted_idx = lengths.sort(descending=True)
        _, orig_idx = sorted_idx.sort()

        # Sort sequences for packing
        idxs_sorted = idxs[sorted_idx]
        emb = self.embedding(idxs_sorted)
        packed = pack_padded_sequence(emb, sorted_lens.cpu(), batch_first=True, enforce_sorted=True)

        # LSTM encoding
        packed_out, _ = self.lstm(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)  # [batch, max_len, hidden]
        out = self.dropout(out)
        out = out[orig_idx]  # restore original order

        # Insert sentinel at end-of-sequence index for each example
        batch_size = out.size(0)
        sentinel_expanded = self.sentinel.expand(batch_size, 1, self.hidden_dim)  # [batch, 1, hidden]

        out_with_sentinel = torch.cat([out, torch.zeros_like(sentinel_expanded)], dim=1)  # [batch, max_len+1, hidden]
        lens = lengths.long().unsqueeze(1).unsqueeze(2).expand(-1, 1, self.hidden_dim)  # [batch, 1, hidden]
        out_with_sentinel = out_with_sentinel.scatter(1, lens, sentinel_expanded)

        return out_with_sentinel  # [batch, seq_len + 1, hidden]

    def forward(self, doc_idxs, doc_mask, q_idxs, q_mask):
        """
        doc_idxs/q_idxs: [batch, seq_len]
        doc_mask/q_mask: [batch, seq_len]
        """
        D = self.encode_sequence(doc_idxs, doc_mask)  # [batch, m+1, hidden]
        Q_prime = self.encode_sequence(q_idxs, q_mask)  # [batch, n+1, hidden]

        # Nonlinear projection: Q = tanh(W * Q′ + b)
        Q = torch.tanh(self.w(Q_prime) + self.b)  # [batch, n+1, hidden]

        return D, Q       

## Small Encoder test

In [ ]:
import torch
import numpy as np

# Dummy vocab
word2idx = {
    "[PAD]": 0, "[UNK]": 1,
    "the": 2, "quick": 3, "brown": 4, "fox": 5, "jumps": 6, "over": 7, "lazy": 8, "dog": 9
}

# Random embedding matrix for vocab (vocab_size x emb_dim)
vocab_size = len(word2idx)
embedding_dim = 50
embedding_matrix = np.random.uniform(-0.1, 0.1, (vocab_size, embedding_dim))

# Dummy inputs
# Document: "the quick brown fox jumps"
# Question: "the fox"
doc_tokens = [2, 3, 4, 5, 6]
q_tokens = [2, 5]

# Padding to max length
doc_max_len = 6
q_max_len = 4
doc_input = [doc_tokens + [0] * (doc_max_len - len(doc_tokens))]  # batch size 1
q_input = [q_tokens + [0] * (q_max_len - len(q_tokens))]

# Masks (1 for real tokens, 0 for padding)
doc_mask = [[1]*len(doc_tokens) + [0]*(doc_max_len - len(doc_tokens))]
q_mask = [[1]*len(q_tokens) + [0]*(q_max_len - len(q_tokens))]

# Convert to tensors
doc_idxs = torch.tensor(doc_input)      # [1, 6]
doc_mask = torch.tensor(doc_mask)       # [1, 6]
q_idxs = torch.tensor(q_input)          # [1, 4]
q_mask = torch.tensor(q_mask)           # [1, 4]

hidden_size = 64
encoder = Encoder(hidden_size, embedding_matrix, 0)

# Run encoder
D, Q = encoder(doc_idxs, doc_mask, q_idxs, q_mask)

# Outputs
print("Document encoding shape:", D.shape)  # [1, m+1, 64]
print("Question encoding shape:", Q.shape)      # [1, n+1, 64]

print("\nSentinel vector (document):", D[0, -1])
print("Sentinel vector (question):", Q[0, -1])


Document encoding shape (with sentinel): torch.Size([1, 6, 64])
Question encoding shape (projected): torch.Size([1, 3, 64])

Sentinel vector (document): tensor([ 1.7383,  0.8497, -1.3716, -1.5591,  0.0837,  0.1320,  1.2961, -1.5653,
         0.3502,  0.5081,  0.9126, -0.6087,  0.0031,  0.3404, -0.7516, -0.1366,
         2.1637, -0.0730,  1.2716,  0.6490, -0.7126, -0.4458,  0.4099, -0.4572,
        -0.2072, -0.8879,  0.2099,  0.3814,  2.4512,  0.1368, -1.0433, -0.3454,
        -1.0600,  1.3232,  0.1469,  0.8049,  0.4266, -0.1950, -1.8880, -1.1639,
         1.2693,  0.3779,  0.1997,  1.1024,  0.9713, -0.2005, -0.3045,  0.4434,
         0.3661,  1.2391,  0.9023,  1.7191, -0.6880, -0.4479,  1.0720,  1.0619,
        -1.1908, -0.0930, -0.0781, -0.3598, -0.9050, -0.7227,  1.1644,  0.5092],
       grad_fn=<SelectBackward0>)
Sentinel vector (question): tensor([ 0.5440, -0.0703, -0.1595, -0.1310,  0.8475, -0.5078,  0.2081,  0.8371,
         0.5305, -0.4063,  0.0593,  0.1951, -0.6979,  0.7548,  0